# Start Dask

In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

/anaconda3/envs/overscripted/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


# Objective

The objective of this notebook is to answer two main questions: 
 - "Are there a set of location domains that always produces a JSON?"

## Conclusion
To answer "Are there a set of location domains that always produces a JSON?", YES, but it counts only to 0.16%, and
about 31% of all the domains can produces both types of values, json and non-json, so, we can say that only 31% of all the domains are capaple of geting JSONs, all the others 69% will never get a JSON value. 

---

There are 11185 different scripts domains, 93.60% of those appear in multiple rows across the dataset. 

Most domains only have one type of value output, either they are json or not-json.
For the domains that have one tipe of output, 99% of the time they get the same keys_md5, but it may not be very accurate because every non-json value has a empty key and thus equal, so, after filtering to valid jsons we have only 19 domains with unique output && valid jsons, out of this, 63% are aways the same json.

In [2]:
DIR = 'sample_0_prep/'

In [3]:
df = dd.read_parquet(DIR + 's0_domains_isJson_jsonKeys_md5_TLD.parquet',
                         engine='pyarrow',
                         columns=['is_json', 'location_domain', 'script_domain', 'keys_md5'])
df.columns

Index(['is_json', 'location_domain', 'script_domain', 'keys_md5'], dtype='object')

---

# Are there a set of location domains that always produces a JSON?


In [4]:
with ProgressBar():
    group_by_script_domain = df.compute().groupby(['script_domain'])
    group_by_script_domain_len = len(group_by_script_domain)
    print("The total number of different {} is {}".format('script_domain', group_by_script_domain_len))

[########################################] | 100% Completed |  3.5s
The total number of different script_domain is 11185


In [5]:
agg = group_by_script_domain.agg({'is_json': ['nunique', 'sum'],
                             'location_domain': ['nunique'],
                             'keys_md5': ['nunique']})

In [6]:
appear_multiple_times = agg['is_json'][group_by_script_domain['is_json'].count() > 1]
appear_multiple_times_len = len(appear_multiple_times)
agg_len = len(agg['is_json'])
print('There are a total of {0}({1:0.2f}%) JSONs that appear in multiple rows'.format(
    appear_multiple_times_len, 
    appear_multiple_times_len*100/agg_len))

There are a total of 10469(93.60%) JSONs that appear in multiple rows


In [7]:
def get_unique(agg, column, title=''):
    agg_len = len(agg[column])
    x = agg[agg[column]['nunique'] == 1]
    x_len = len(x)
    print(title + '{0} ({1:0.2f}%) unique {2},\n{3} ({4:0.2f}%) multiple {2}'.format(
        x_len,
        x_len*100/agg_len,
        column, 
        agg_len - x_len,
        (agg_len - x_len) * 100 / agg_len
    ))
    return x

In [8]:
unique_is_json = get_unique(agg, 'is_json', 'JSON data:\n')
unique_json_key = get_unique(unique_is_json, 'keys_md5', '\nKEYS data: out of the unique jsons\n')
#it may not be very accurate because every non-json value has a empty key and thus equal

JSON data:
7697 (68.82%) unique is_json,
3488 (31.18%) multiple is_json

KEYS data: out of the unique jsons
7690 (99.91%) unique keys_md5,
7 (0.09%) multiple keys_md5


In [9]:
#Filter to only valid jsons
unique_is_json_jsons = unique_is_json[unique_is_json['is_json']['sum'] > 0]
print("There are only {} domains with unique valid json output".format(len(unique_is_json_jsons)))
unique_key_jsons = get_unique(unique_is_json_jsons, 'keys_md5', '\nKEYS data: out of the unique valid jsons\n')

There are only 19 domains with unique valid json output

KEYS data: out of the unique valid jsons
12 (63.16%) unique keys_md5,
7 (36.84%) multiple keys_md5
